In [ ]:
#!pip install pygad

model inspired and modified from http://uhurumkate.blogspot.com/2018/06/tiny-model-for-mnist-dataset.html

You're highly encouraged to use a GPU as it's going to spent most resources on predicting and calculating acc. Please also note that pygad library only use cpu paralleal in some cases. <br>
P.S.2 It takes painfully long to converage. About an hour per 10% increase

In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import losses, datasets, layers, optimizers, Sequential, metrics
import pygad.kerasga
import numpy
import pygad

In [ ]:
numpy.random.seed(123)
acc_meter = tf.keras.metrics.Accuracy()

#Instead of training a entire dataset, do small batches.
sgd_like = False

#Introduce new population when stucked
introduce_new_when_stucked = False

# dataset

In [ ]:
#load mnist dataset
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
#normalize data
X_train = 2*tf.convert_to_tensor(X_train, tf.float32)/255.-1
#[b, 28, 28] -> [b, 28, 28, 1]
X_train = tf.reshape(X_train, (-1, 28,28,1))
X_test = 2*tf.convert_to_tensor(X_test, tf.float32)/255.-1
X_test = tf.reshape(X_test, (-1, 28,28,1))
y_train = tf.convert_to_tensor(y_train, tf.int32)
#[b] -> [b, 10]
y_train = tf.one_hot(y_train, 10)
y_test = tf.convert_to_tensor(y_test, tf.int32)
y_test = tf.one_hot(y_test, 10)

# Model

We want a very small model for the increase in param will increase the cycles GA needs to run </ba>
Model parameter = 1810 with acc ~ 96% on both train/testing sets

In [ ]:
model = Sequential([
#input layer
layers.Convolution2D(15, (3, 3), activation='relu', input_shape=(28,28,1)),
layers.BatchNormalization(momentum=0.1),
layers.Dropout(0.1),
layers.AveragePooling2D(2),
#convo layers1
layers.Convolution2D(10,(1,1), activation='relu'),
layers.BatchNormalization(momentum=0.1),
layers.Dropout(0.1),
#convo layers2
layers.AveragePooling2D(2),
layers.Convolution2D(5,(3,3), activation='relu'),
layers.BatchNormalization(momentum=0.1),
layers.Dropout(0.1),
#convo layers3
layers.Convolution2D(10,(1,1), activation='relu'),
layers.Dropout(0.1),
#fully connected layers
layers.Dense(5, activation='relu'),
layers.Flatten(),
#output layer
layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )

#model.fit(X_train, y_train, 
#          batch_size=8, verbose=1,shuffle=True,
#          validation_data=(X_test, y_test))

Network accuracy and error

In [ ]:
acc_meter.update_state(tf.argmax(model.predict(X_test), axis=1), tf.argmax(y_test, axis=1))
print("Accuracy on testing set:{acc}%".format(acc=acc_meter.result().numpy()*100))
print("Error rate on testing set:{err}%".format(err=(1-acc_meter.result().numpy())*100))
acc_meter.update_state(tf.argmax(model.predict(X_train), axis=1), tf.argmax(y_train, axis=1))
print("Accuracy on training set:{acc}%".format(acc=acc_meter.result().numpy()*100))
print("Error rate on training set:{err}%".format(err=(1-acc_meter.result().numpy())*100))

loss

In [ ]:
#cce = tf.keras.losses.CategoricalCrossentropy()
#cce(y_train, model.predict(X_train))

In [ ]:
#we don't need one-hot to train. To compute tf.argmax each loop is a waste of resources
y_train = tf.argmax(y_train, axis=1)

Perpare batches if we were doing SGD-like method. <br>
Some data are removed if size of dataset / batch_size is not an integer

In [ ]:
X_train_sgd = []
y_train_sgd = []
if sgd_like == True:
  batch_size = 3000
  if len(X_train) % batch_size == 0:
    X_train_sgd = tf.reshape(X_train, [-1, batch_size, 28, 28, 1])
    y_train_sgd = tf.reshape(y_train, [-1, batch_size])
  else:
    len_remove = len(X_train) % batch_size
    X_train_sgd = tf.reshape(X_train[:-len_remove], [-1, batch_size, 28, 28, 1])
    y_train_sgd = tf.reshape(y_train[:-len_remove], [-1, batch_size])

# Hyper param

In [ ]:
#num cycles
num_generations = 1000

#num solutions
num_solution=10

#the following two param have no effect when introduce_new_when_stucked = false
#Maxium cycles stuck before introducing new population to the pool.
max_cycles_stucked = 400
#number of population to be reinitlized. Much be >0 and  <= num_generations.
num_reintroduce = 3

#number of cycles you want on a single batch. Only works when sgd_like = True
epoch = 100

#70 % parents mating
num_parents_mating = int(num_solution * .7)
parent_selection_type = "sss"
crossover_type = "uniform"
mutation_type="adaptive"
mutation_num_genes=(90, 80)
keep_parents = 3

# Fitness 

In [ ]:
i = 0
cycles_stucked = 0
tmp = 0

def fitness_func(solution, sol_idx):

    global keras_ga, model, i, cycles_stucked ,X_train, y_train
    solution_fitness = 0
    model_weights_matrix = pygad.kerasga.model_weights_as_matrix(model=model,
                                                                 weights_vector=solution)

    model.set_weights(weights=model_weights_matrix)
    
    #introduce new population to the pool if reached max cycles stucked. Max_cycles_stucked -> 0.
    if introduce_new_when_stucked:
        if cycles_stucked == max_cycles_stucked:
            ga_instance.population[:3] = tf.random.truncated_normal(ga_instance.population[:3].shape)
            cycles_stucked = 0

    #solution_fitness = 1/cce(model.predict(X_train), y_train).numpy()
    
    if sgd_like == True:
        if i % (1 * epoch) == 0:
            #selec.t random data from reshaped dataset to train
            k = numpy.random.randint(0, len(X_train_sgd)-1)
            #update acc_meter
            acc_meter.update_state(tf.argmax(model.predict(X_train_sgd[k]), axis=1), y_train_sgd[k])
            solution_fitness = acc_meter.result().numpy() * 100
            acc_meter.reset_states()
    else:
        acc_meter.update_state(tf.argmax(model.predict(X_train), axis=1), y_train)
        solution_fitness = acc_meter.result().numpy() * 100
        acc_meter.reset_states()
    return solution_fitness

In [ ]:
def callback_generation(ga_instance):
    global i, cycles_stucked, tmp
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    i += 1
    if ga_instance.best_solution()[1] == tmp:
        cycles_stucked += 1
    else:
        cycles_stucked = 0
    tmp = ga_instance.best_solution()[1]
    


keras_ga = pygad.kerasga.KerasGA(model=model,
                                 num_solutions=num_solution)


# Prepare the PyGAD parameters. Check the documentation for more information: https://pygad.readthedocs.io/en/latest/README_pygad_ReadTheDocs.html#pygad-ga-class
# Use the pre-existing model weight as init population. 
initial_population = numpy.tile(pygad.kerasga.model_weights_as_vector(model), (num_solution, 1))


ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       keep_parents=keep_parents,
                       mutation_type=mutation_type,
                       parent_selection_type=parent_selection_type,
                       crossover_type=crossover_type,
                       mutation_num_genes=mutation_num_genes,
                       on_generation=callback_generation)
ga_instance.run()

In [ ]:

# After the generations complete, some plots are showed that summarize how the outputs/fitness values evolve over generations.
ga_instance.plot_fitness(title="PyGAD & Keras - Iteration vs. Fitness", linewidth=4)

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

# Fetch the parameters of the best solution.
best_solution_weights = pygad.kerasga.model_weights_as_matrix(model=model,
                                                              weights_vector=solution)
model.set_weights(best_solution_weights)

acc_meter.update_state(tf.argmax(model.predict(X_test), axis=1), tf.argmax(y_test, axis=1))
print("Accuracy on testing set:{acc}%".format(acc=acc_meter.result().numpy()*100))
print("Error rate on testing set:{err}%".format(err=(1-acc_meter.result().numpy())*100))

acc_meter.update_state(tf.argmax(model.predict(X_train), axis=1), y_train)
print("Accuracy on training set:{acc}%".format(acc=acc_meter.result().numpy()*100))
print("Error rate on training set:{err}%".format(err=(1-acc_meter.result().numpy())*100))
